In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
import matplotlib.cm as cm
import matplotlib.colors as colors


In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [4]:
df = pd.read_html(URL)
df = df[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [5]:
df = df[df["Borough"] != "Not assigned"]

In [6]:
for row in df.iterrows():
    index = row[0]
    borough = row[1][1]
    neighbourhood = row[1][2]
    if neighbourhood == "Not assigned":
        neighbourhood = borough
    df['Neighbourhood'].loc[index] = neighbourhood


<ipython-input-6-2242b93387fd>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Neighbourhood'].loc[index] = neighbourhood


In [7]:
df.set_index('Postal Code')

,Borough,Neighbourhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...
M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
M4Y,Downtown Toronto,Church and Wellesley
M7Y,East Toronto,"Business reply mail Processing Centre, South C..."


In [8]:
df.shape[0]

103

In [9]:
cordinatets_df = pd.read_csv(r'C:\Users\sahan\Downloads\Geospatial_Coordinates.csv', index_col='Postal Code')

In [10]:
cordinatets_df

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476
...,...,...
M9N,43.706876,-79.518188
M9P,43.696319,-79.532242
M9R,43.688905,-79.554724


In [11]:
df = pd.merge(df, cordinatets_df, on='Postal Code',how='left')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [12]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [13]:
from geopy.geocoders import Nominatim

In [14]:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [16]:
! pip install folium
import folium


In [17]:
df_grouped = df.groupby('Borough').mean()

In [18]:
import random
def gen_colors(num):
    random.seed(1234)
    return ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)]) for i in range(num)]


In [19]:
df_grouped['MarkerColor'] = pd.Series(gen_colors(df_grouped.shape[0]), index=df_grouped.index)

In [20]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10, tiles='Stamen Terrain')

for borough, neighbor, lat, lng in zip(df['Borough'], df['Neighbourhood'], df['Latitude'], df['Longitude']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=df_grouped.loc[borough,'MarkerColor'],
        popup=f"{borough}, {neighbor}",
        fill = True,
        fill_color=df_grouped.loc[borough,'MarkerColor'],
        fill_opacity=0.8
    ).add_to(toronto_map)

toronto_map


In [25]:
import config

In [26]:
import config
config.CLIENT_ID = "F5LKCXZIMXVSFZJNXKRC00RHRT4440ESDKNCBE1SWQZNZLPS"
config.CLIENT_SECRET = "IX0DMTBNNBT5NIF421DGULYQ4MX0ICMMM5M3AQASO5HSYVJQ"
VERSION = '20180605'
LIMIT = 100

In [27]:
selected_borough = df.groupby('Borough').count().sort_values(by='Neighbourhood', ascending=False).index[0]


In [28]:
selected_borough

'North York'

In [29]:
selected_df =  df[df['Borough'] == selected_borough]

In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
       
        import config
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            config.CLIENT_ID, 
            config.CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
       
        results = requests.get(url).json()["response"]['groups'][0]['items']
        

        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
ny_venues = getNearbyVenues(names=selected_df['Neighbourhood'],
                                   latitudes=selected_df['Latitude'],
                                   longitudes=selected_df['Longitude']
                                  )


Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [32]:
ny_venues.shape

(240, 7)

In [33]:
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 102 uniques categories.


In [34]:
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")


ny_onehot['Neighbourhood'] = ny_venues['Neighbourhood'] 


fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
ny_grouped = ny_onehot.groupby('Neighbourhood').mean().reset_index()
ny_grouped

,Neighbourhood,Accessories Store,Airport,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.095238,...,0.047619,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.0000,0.045455,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.045455,0.000000,0.045455,0.000000,0.000000,0.000000,0.000000,0.000000
3,Don Mills,0.000000,0.0000,0.000000,0.043478,0.000000,0.043478,0.0000,0.000000,0.000000,...,0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Downsview,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.0625,0.000000,0.062500,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Fairview, Henry Farm, Oriole",0.000000,0.0000,0.014286,0.000000,0.000000,0.014286,0.0000,0.028571,0.028571,...,0.000000,0.014286,0.000000,0.014286,0.000000,0.014286,0.014286,0.014286,0.000000,0.042857
6,Glencairn,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Hillcrest Village,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.2000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Humber Summit,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Humberlea, Emery",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']


columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))


neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Mobile Phone Shop,Sandwich Place,Gas Station,Diner,Ice Cream Shop,Deli / Bodega,Chinese Restaurant,Middle Eastern Restaurant
1,Bayview Village,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
2,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Greek Restaurant,Grocery Store,Indian Restaurant,Juice Bar,Liquor Store,Locksmith,Comfort Food Restaurant
3,Don Mills,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant,Italian Restaurant
4,Downsview,Grocery Store,Park,Bank,Liquor Store,Hotel,Shopping Mall,Home Service,Baseball Field,Business Service,Gym / Fitness Center


In [38]:
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighbourhood', 1)


from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)


kmeans.labels_[0:10]


array([4, 4, 4, 4, 4, 4, 4, 4, 3, 2])

In [39]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = selected_df

ny_merged = ny_merged.merge(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
ny_merged.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Food & Drink Shop,Park,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,4,Clothing Store,Women's Store,Vietnamese Restaurant,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Gift Shop,Accessories Store,Supermarket
3,M3B,North York,Don Mills,43.745906,-79.352188,4,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant,Italian Restaurant
4,M3C,North York,Don Mills,43.725900,-79.340923,4,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant,Italian Restaurant


In [40]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Food & Drink Shop,Park,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
16,North York,0,Construction & Landscaping,Bakery,Park,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Convenience Store,Cosmetics Shop,Deli / Bodega
21,North York,0,Convenience Store,Park,Women's Store,Discount Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Cosmetics Shop,Deli / Bodega,Department Store


In [42]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,1,Martial Arts School,Women's Store,Discount Store,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [43]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2,Baseball Field,Women's Store,Dog Run,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


In [44]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,3,Furniture / Home Store,Pizza Place,Women's Store,Diner,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


In [45]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,4,Coffee Shop,Pizza Place,Hockey Arena,Portuguese Restaurant,Intersection,Diner,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,North York,4,Clothing Store,Women's Store,Vietnamese Restaurant,Boutique,Coffee Shop,Event Space,Furniture / Home Store,Gift Shop,Accessories Store,Supermarket
3,North York,4,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant,Italian Restaurant
4,North York,4,Gym,Beer Store,Coffee Shop,Japanese Restaurant,Caribbean Restaurant,Clothing Store,Chinese Restaurant,Café,Dim Sum Restaurant,Italian Restaurant
5,North York,4,Park,Bakery,Pizza Place,Pub,Japanese Restaurant,Food Truck,Dessert Shop,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping
6,North York,4,Dog Run,Athletics & Sports,Mediterranean Restaurant,Pool,Golf Course,Fried Chicken Joint,Food Truck,Comfort Food Restaurant,Construction & Landscaping,Convenience Store
7,North York,4,Coffee Shop,Bank,Mobile Phone Shop,Sandwich Place,Gas Station,Diner,Ice Cream Shop,Deli / Bodega,Chinese Restaurant,Middle Eastern Restaurant
8,North York,4,Clothing Store,Coffee Shop,Fast Food Restaurant,Juice Bar,Japanese Restaurant,Women's Store,Food Court,Shoe Store,Bakery,Bank
9,North York,4,Furniture / Home Store,Miscellaneous Shop,Caribbean Restaurant,Metro Station,Massage Studio,Bar,Coffee Shop,Women's Store,Dim Sum Restaurant,Construction & Landscaping
10,North York,4,Chinese Restaurant,Café,Bank,Japanese Restaurant,Women's Store,Discount Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
